In [2]:
import django
import sys, os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
import numpy as np

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from twitter.models import *
import parliament.models as pms

from django.db.models import Sum, Count
from django.db.models.fields import DateField
from django.db.models.functions import Cast
import textwrap as tw
import datetime
import random

import scoping.models as sm
import igraph
#import mpld3

from adjustText import adjust_text

print(igraph.__version__)



p = sm.Project.objects.get(pk=185)

0.7.1


In [13]:
# Get tweets related to the project
searches = TwitterSearch.objects.filter(project=p).exclude(string__in=["#ETS","#EUETS"])

ttweets = set(Status.objects.filter(
    searches__in=searches,
    text__isnull=False,
    retweeted_status__isnull=True
).values_list('id',flat=True))

print(len(ttweets))


113155


In [14]:
# Take a sample of the texts
texts = Status.objects.filter(
    id__in=random.sample(ttweets, 20000)
).values_list('text',flat=True)


# Stem and vectorize the texts (need to remove @s?)
from parliament.tasks import * 
stemmer = SnowballStemmer("german")
tokenizer = german_stemmer()
stopword_list = [stemmer.stem(t) for t in stopwords.words("german")]


vectorizer = CountVectorizer(max_df=0.8,
                             min_df=5,
                             max_features=10000,
                             tokenizer=tokenizer,
                             stop_words=stopword_list)

tfidf = vectorizer.fit_transform(texts)

/home/galm/software/django/tmv/venv/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['gew'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [22]:
K = 10

model = NMF(
    n_components=K, random_state=1,
    alpha=0.01, l1_ratio=.91,
    init='nndsvd'
).fit(tfidf)

dtm = model.transform(tfidf)

In [23]:


tfidf_feature_names = vectorizer.get_feature_names()
def display_topics(model, feature_names, no_top_words, dtm, texts):
    for topic_idx, topic in enumerate(model.components_):
        tdocs = dtm[:,topic_idx]
        print(f"\n##############################################\n### Topic {topic_idx}")
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        print("\n")
        print("\n\n".join([texts[i]
                        for i in tdocs.argsort()[:-5 - 1:-1]]))
        
        
no_top_words = 10
display_topics(model, tfidf_feature_names, no_top_words, dtm, list(texts))


##############################################
### Topic 0
costeu polit klima via klimawandel deutschland kommt grun gut burg


Für die CO2-Steuer! Für die CO2-Steuer! Für die CO2-Steuer! Für die CO2-Steue..... !!!!!!

»Deutschland: Warum die Politik zu blöd für eine CO2-Steuer ist «       https://www.spiegel.de/wirtschaft/soziales/deutschland-warum-die-politik-zu-bloed-fuer-eine-co2-steuer-ist-a-1266619.html …      »Kampf um CO2-Steuer  Die Politik ist zu dämlich für den Klimaschutz  Die bizarre deutsche Debatte um die CO2-Steuer zeigt:  Der…  https://social.tchncs.de/@imageschaden/102257983611530886 …

Folgende Maßnahmen kommen in Deutschland in Frage: CDU&gt;&gt;Erhöhung der CO2-Steuer SPD&gt;&gt;Erhöhung der CO2-Steuer+ Vermögenssteuer+Aktiensteuer
Grüne&gt;&gt;Erhöhung der CO2-Steuer + Einführung der Vermögenssteuer +Aktiensteuer +Dieselfahrverbot +Veggie-Day https://t.co/aYhmYvtMt3 via @welt

🌏 Regierungsberater fordern CO2-Steuer: Wenige Tage vor Beginn der Internationalen Uno-

Wir haben schon lange eine CO2-Steuer. Ich erinnere an die KFZ-Steuer. Aber wen interessiert das schon. Schließlich gehts um die Rettung der Welt. Theoretisch. Also eigentlich gehts nur ums Geld. Aber wie gesagt, wen interessiert das schon. 🙈  pic.twitter.com/NzkQ6SEX5K

Schön was Oettinger sagt, wäre erst Recht schön wenn das möglich wäre. Die Wissenschaft sagt aber was anderes. Zum Probieren haben wir nicht mehr genug Zeit aber z.B. die Einführung einer CO2 Steuer hat in GB schon Erfolge gebracht, warum wird sich noch immer dagegen gewährt?

Wir haben doch schon sehr viel Erneuerbare Energie ! Das muss der Bürger auch mitmachen und bezahlen können, wir haben JETZT schon die teuersten Strompreise und zahlen am meisten Steuern mit Belgien, in Europa ! Wir haben schon ne ÖKOSTEUER, wir brauchen keine CO2 STEUER !

CO2 Steuer?  Liebe Freunde, habt ihr wirklich Lust auf noch ne Steuer?  Wir zahlen schon so viele Steuern - und Soli / Ökosteuer usw...  Jetzt wird doch schon an der Tankstell